# fMRI BOLD analysis

New default notebook for all single scan analyses, for both fMRI-only and simultaneous calcium/fMRI.

**Preparation for calcium/fMRI:**

- Copy the *regressorX_married.1D* files in the corresponding raw scan folder (e.g. subjectx/8/,  subjectx/9/  etc.). Will be recognized automatically.

- If no .1D file is present, only stim-evoked GLM is performed.

In [1]:
import os, sys, pathlib, shutil, glob
mymodules = os.fspath(pathlib.Path.cwd().joinpath('../CustomModules'))
if mymodules not in sys.path:
    sys.path.append(mymodules)
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import importlib
import helperfunctionsfMRI as fh
import MRIhelperfunctions as h
import calciumGroupFunctions as g

In [8]:
fh = importlib.reload(fh)
h = importlib.reload(h)
g = importlib.reload(g)

# Settings

In [2]:
paradigm = {}
folders = {}
averaging = {}

paradigm['TR'] = 1 # in seconds

paradigm['initial_cutoff'] = 30 # number of volumes to remove (in absence of dummy scans).

blurFWHM = 0.3 # In [mm]. Rule-of-thumb: 1.5x the voxel resolution.

highpass = 240 # period [s]. Any signal slower than this will be removed. Should be at least 2x inter-stim. interval


# --- For averaging of blocks:

averaging['baseline'] = 20 # baseline to consider before each block

averaging['time'] = 60 # how many seconds to display after end of stim_block


#======================================================================
folders['animal'] = '/mnt/c/Users/felix/Documents/exampledata_python/conversion_problems'

folders['scanNumber'] = '6844_E15'
#======================================================================
# Data folder can contain either the original Bruker raw data, or BRIK file from earlier conversions.

# --- EPI Template for coregistration ---
folders['template'] = '/mnt/c/Users/felix/Documents/exampledata_python/conversion_problems/templates/TEMPLATE.nii'

#Use this scan as new Template (warning, will override old template):
createNewTemplate = False

# --- Optional: ROI file:
folders['rois'] = '/mnt/c/Users/felix/Documents/exampledata_python/conversion_problems/templates/ROI3.nii'



# --- Define the stimulation paradigm (all units in seconds):
paradigm['baseline'] = 120
paradigm['stim_duration'] = 8
paradigm['Nblocks'] = 10
paradigm['ISI'] = 90    #inter-stimulus interval (on+off period)
paradigm['frequency'] = 4


paradigm['stim_times'], paradigm['stim_onsets'], paradigm['stim_times_all'], paradigm['singleblock'] = fh.CreateStimParadigm(paradigm)

stimulus times: [120, 210, 300, 390, 480, 570, 660, 750, 840, 930]


In [3]:
# this creates also the 'analysis' and 'scan' folders
folders = h.check_and_convert_single(folders)

No 2dseq file found. Check if it was renamed or if /mnt/c/Users/felix/Documents/exampledata_python/conversion_problems/6844_E15 is really the main folder of that animal.
using /mnt/c/Users/felix/Documents/exampledata_python/conversion_problems/6844_E15/E15+orig.BRIK.Z
3dAFNItoNIFTI -prefix /mnt/c/Users/felix/Documents/exampledata_python/conversion_problems/analysis/6844_E15/X6844_E15P1.nii /mnt/c/Users/felix/Documents/exampledata_python/conversion_problems/6844_E15/E15+orig.BRIK.Z
replaceHDR function ran through


# Data Preprocessing

In [4]:
folders['cut'] = fh.initial_cutoff(folders['raw'], paradigm['initial_cutoff'])

# takes several minutes
folders['coreg'] = h.coreg_epi_to_template(folders['template'], folders['cut'], createNewTemplate)

3dTcat -overwrite -prefix /mnt/c/Users/felix/Documents/exampledata_python/conversion_problems/analysis/6844_E15/single_timepoint.nii /mnt/c/Users/felix/Documents/exampledata_python/conversion_problems/analysis/6844_E15/data_cut.nii'[0]'
python2.7 /home/felix/abin/align_epi_anat.py -dset2to1 -dset1 /mnt/c/Users/felix/Documents/exampledata_python/conversion_problems/templates/TEMPLATE.nii -dset2 /mnt/c/Users/felix/Documents/exampledata_python/conversion_problems/analysis/6844_E15/single_timepoint.nii -child_dset2 /mnt/c/Users/felix/Documents/exampledata_python/conversion_problems/analysis/6844_E15/data_cut.nii -dset1_strip None -dset2_strip None -overwrite -big_move -volreg on -volreg_method 3dvolreg -volreg_opts '-Fourier -zpad 1 -dfile dfile.1D' -Allineate_opts '-maxrot 10 -maxshf 3 -conv 0.005 -twofirst -twoblur 0.8 -source_automask+2 -final wsinc5' -tshift on -tshift_opts '-tzero 0 -quintic' -dset2_base 0 -volreg_base 0 -suffix .volreg -cost ls


### Check for motion

In [5]:
frame = pd.read_csv(os.path.join(folders['scan'],'dfile.1D'), sep=r'\s{2,}', engine='python')

%matplotlib qt
# frame.plot(subplots=True, layout=(9,1))
frame.iloc[:,1:7].plot(subplots=True, layout=(6,1))

array([[<matplotlib.axes._subplots.AxesSubplot object at 0x7f1e46c2bcf8>],
      dtype=object)

High-pass filter, smoothing

In [6]:
folders['cleaned'] = fh.filter_blur(folders['coreg'], highpass, blurFWHM)

# GLM Analyses

In [9]:
# 1. Standard, using only stim. paradigm
fh.Deconvolve(folders['cleaned'],paradigm) # standard stim_evoked


# 2. Check if regressors present, use both in GLM f-test.
calcium_regressors = g.get_filepaths(folders['raw_folder'], fileending='*married.1D', subfolders=False)
if calcium_regressors:
    fh.Deconvolve_calcium_2ch(folders['cleaned'], paradigm, calcium_regressors)

3dDeconvolve -input /mnt/c/Users/felix/Documents/exampledata_python/conversion_problems/analysis/6844_E15/CleanedData.nii -polort 0 -nodmbase -overwrite -num_stimts 1 -stim_times 1 '1D: 120 210 300 390 480 570 660 750 840 930' 'SPMG3(8)' -stim_times_subtract 30 -stim_label 1 stim -iresp 1 HRF_SPM_el -fout -tout -x1D XSPM.xmat.1D -xjpeg XSPM.jpg -fitts fittsSPM_ -errts errtsSPM -bucket statsSPM_ -cbucket regcoeffsSPM
files found: 
 ['/mnt/c/Users/felix/Documents/exampledata_python/conversion_problems/6844_E15/regressor1_married.1D', '/mnt/c/Users/felix/Documents/exampledata_python/conversion_problems/6844_E15/regressor2_married.1D']
3dDeconvolve -input /mnt/c/Users/felix/Documents/exampledata_python/conversion_problems/analysis/6844_E15/CleanedData.nii -polort 0 -nodmbase -overwrite -num_stimts 2 -stim_times_AM1 1 /mnt/c/Users/felix/Documents/exampledata_python/conversion_problems/6844_E15/regressor1_married.1D 'SPMG3(0.01)' -stim_times_AM1 2 /mnt/c/Users/felix/Documents/exampledata_pyt

# ROI Analysis

Can be run separately on previously analyzed datasets, but first execute all cells under "Settings" above

In [10]:
folders['cleaned'] = fh.find_cleaned_data(folders)
folders['roidata'] = h.extract_roi(folders['cleaned'],folders['rois'])

roi_df = pd.read_csv(folders['roidata'], sep='\t')
del roi_df['File'], roi_df['Sub-brick']

Plotting whole time course

In [11]:
%matplotlib qt

plt.figure()
for rois in roi_df.columns:
    plt.plot(roi_df[rois])
_ = [plt.axvline(_p, alpha=0.05, color='red') for _p in paradigm['stim_times_all']]
_ = plt.legend(roi_df.columns)

Plotting averages across stimulus blocks

In [12]:
%matplotlib qt

roimeans = fh.averageblocks(roi_df, paradigm, averaging)

plt.figure()
_ = [plt.axvline(_p, alpha=0.05, color='red') for _p in paradigm['singleblock']]
plt.axhline(0,color='k')
plt.plot(roimeans)
_ = plt.legend(roi_df.columns)